In [ ]:
"""
Module: MODIS NDVI Processing Script

Script Functionality:
This script processes MODIS NDVI data over a specified region of interest (ROI) and date range. 
It retrieves the NDVI, EVI, and NIRv values, applies quality assurance (QA) checks, filters valid pixels, 
calculates the median and 90th percentile of NDVI, and exports the results to Google Drive.

Input Parameters:
A feature collection representing the region of interest (ROI).

Dependencies:
Earth Engine Python API :pip install earthengine-api (for accessing Earth Engine data).
Geemap :pip install geemap (for interactive mapping with Earth Engine).

Version Information:
Version: v1.0
Change Log: October 08 2024.
"""
import ee
import geemap
# Initialize the Earth Engine API
ee.Authenticate()
ee.Initialize()

# Define region of interest (ROI)
roi = ee.FeatureCollection('projects/ee-meiyang150269/assets/boundary')
# roi = ee.FeatureCollection('users/your_username/your_table')
# Define date range
start_date = '2022-01-01'
end_date = '2023-01-01'

# Load MODIS ImageCollection and select bands
MOD13A2 = ee.ImageCollection('MODIS/061/MOD13A1')\
            .filter(ee.Filter.date(start_date, end_date))\
            .select(['NDVI', 'EVI', 'sur_refl_b01', 'SummaryQA'])\
            .map(lambda img: img.clip(roi))\
            .map(lambda image: image.addBands(
                image.select('NDVI').subtract(800).max(0).multiply(
                    image.select('sur_refl_b01').max(0)
                ).toFloat().divide(10000).divide(10000).rename('NIRv')
            ))

# Define QA function
def CheckQA(image):
    qa = image.select('SummaryQA')
    mask2 = qa.eq(0)
    return image.updateMask(mask2)

# Apply QA function and select bands
NIRv_afterQAcheck = MOD13A2.map(CheckQA).select(['NDVI', 'EVI', 'NIRv'])

# Calculate count of valid pixels
count = NIRv_afterQAcheck.select('NDVI').reduce(ee.Reducer.count()).rename('count')
NIRv_afterQAcheck = NIRv_afterQAcheck.map(lambda img: img.addBands(count))

# Filter based on count
def CheckCount(image):
    count = image.select('count')
    mask = count.gte(14)
    return image.updateMask(mask)

NIRv_CheckQAandCount = NIRv_afterQAcheck.map(CheckCount).select(['NDVI', 'EVI', 'NIRv'])

# Reduce for median or min (change according to your requirement)
NIRv_CheckQAandCount_Median = NIRv_CheckQAandCount.select('NDVI').reduce(ee.Reducer.min())
NIRv_Median = MOD13A2.select('NDVI').reduce(ee.Reducer.min())

# Create final output by unmasking
NIRv_Median_Output = NIRv_CheckQAandCount_Median.unmask(NIRv_Median).multiply(0.0001)

# Calculate 90th percentile
NIRv_CheckQAandCount_90Per = NIRv_CheckQAandCount.reduce(ee.Reducer.percentile([90]))
NIRv_90Per = MOD13A2.select('NDVI').reduce(ee.Reducer.percentile([90]))
NIRv_90Per_Output = NIRv_CheckQAandCount_90Per.unmask(NIRv_90Per)

# Export the final images
task1 = ee.batch.Export.image.toDrive(
    image=NIRv_Median_Output,
    description='NDVIMin2022',
    folder='NDVI',
    region=roi.geometry().bounds().getInfo()['coordinates'],
    crs='EPSG:4326',
    scale=500,
    maxPixels=1e13
)

task2 = ee.batch.Export.image.toDrive(
    image=NIRv_90Per_Output,
    description='NDVI90Per2022',
    folder='NDVI',
    region=roi.geometry().bounds().getInfo()['coordinates'],
    crs='EPSG:4326',
    scale=500,
    maxPixels=1e13
)

# Start the tasks
task1.start()
task2.start()

print("Export tasks started.")

# End of the script

Export tasks started.


resample

In [ ]:
"""
Module: NDVI Raster Resampling Script

Script Functionality:
This script resamples NDVI raster data based on a reference raster file. 
It reads NDVI raster files from a specified input folder, resamples them to match the 
resolution and projection of the reference raster, and saves the resampled output files 
to a designated folder.

Input Parameters:
A reference raster file (.tif) with the desired target resolution and projection.
A folder containing NDVI raster files (.tif) that need to be resampled.

Dependencies:
GDAL : pip install gdal (for raster processing and reprojection)
Rasterio: pip install rasterio (optional, for additional raster operations)
NumPy : pip install numpy(for numerical array operations if needed)


Version Information:
Version: v1.0
Change Log: October 08 2024.
"""

from osgeo import gdal
import rasterio
from rasterio.enums import Resampling
import os
import numpy as np

referencefilepath = r'D:\ghana\0.005.tif'
inputfolderpath = r'D:\ghana\vi\NDVI\0.005'
inputfilepaths = []
outputfolderpath = r'D:\ghana\vi\NDVI\0.005\Resample'
outputfilepaths = []

# Get projection information
referencefile = gdal.Open(referencefilepath, gdal.GA_ReadOnly)
rfproj = referencefile.GetProjection()
bandreferencefile = referencefile.GetRasterBand(1)
rfTrans = referencefile.GetGeoTransform()
rfwidth = referencefile.RasterXSize
rfheight = referencefile.RasterYSize
nbands = referencefile.RasterCount

# Read data
file_list = os.listdir(inputfolderpath)
for filename in file_list:
    if filename.endswith('.tif'):
        inputpath = os.path.join(inputfolderpath, filename)
        outputpath = os.path.join(outputfolderpath, '0.005_' + filename)
        inputfilepaths.append(inputpath)
        outputfilepaths.append(outputpath)

print(inputfilepaths)

# Resampling
def ReprojectImages(inputFilePaths, outputFilePaths):
    for i in range(len(inputFilePaths)):
        inputfilePath = inputFilePaths[i]
        outputfilePath = outputFilePaths[i]
        # Get input raster file information
        inputrasfile = gdal.Open(inputfilePath, gdal.GA_ReadOnly)
        inputProj = inputrasfile.GetProjection()
        # Create resampled output file (set projection and six parameters)
        driver = gdal.GetDriverByName('GTiff')
        output = driver.Create(outputfilePath, rfwidth, rfheight, nbands, bandreferencefile.DataType)
        output.SetGeoTransform(rfTrans)
        output.SetProjection(rfproj)
        # Parameters: input dataset, output file, input projection, reference projection, resampling method (Nearest Neighbor, Bilinear, Cubic Convolution, etc.), callback function
        gdal.ReprojectImage(inputrasfile, output, inputProj, rfproj, gdal.GRA_NearestNeighbour)

ReprojectImages(inputfilepaths, outputfilepaths)

# End of the script

In [ ]:
"""
Module: NIRv Raster Resampling Script

Script Functionality:
This script resamples Near Infrared Vegetation Index (NIRv) raster data files (.tif) located in a specified input folder. 
It processes each raster file to reduce its resolution by averaging values based on a defined scale factor (5), 
resulting in a new resolution of 0.05. The resampled raster files are saved in a designated output folder with 
updated filenames to indicate the new resolution.

Input Parameters:
input_folder: A folder containing NIRv raster files (.tif) to be resampled.
output_folder: A folder where the resampled NIRv raster files will be saved.

Dependencies:
GDAL: pip install gdal (for raster processing and reprojection)
Rasterio: pip install rasterio (for raster file operations)
NumPy: pip install numpy (for numerical array operations)

Version Information:
Version: v1.0
Change Log: November 03 , 2024.
"""

import os
import rasterio
from rasterio.enums import Resampling
import numpy as np

# Set input and output folder paths
input_folder = r"D:\ghana\11.3\NIRv"  # Replace with your input folder path (NDVI NIRv EVI)
output_folder = r"D:\ghana\11.3\NIRv1"  # Replace with your output folder path (NDVI NIRv EVI)

# Ensure the output folder exists
os.makedirs(output_folder, exist_ok=True)

# Process all TIFF files in batch
for filename in os.listdir(input_folder):
    if filename.endswith('.tif'):
        input_file = os.path.join(input_folder, filename)  # Construct full path
        print(f"Processing file: {input_file}")
        
        with rasterio.open(input_file) as src:
            data = src.read(1)  # Read the first band
            profile = src.profile
            
            # Set new resolution (0.05) and calculate scale factor
            scale_factor = 5  # Aggregation factor from 0.01 to 0.05
            new_height = data.shape[0] // scale_factor
            new_width = data.shape[1] // scale_factor

            # Mean aggregation resampling
            resampled_data = data[:new_height * scale_factor, :new_width * scale_factor].reshape(
                new_height, scale_factor, new_width, scale_factor).mean(axis=(1, 3))

            # Update profile
            profile.update({
                "height": new_height,
                "width": new_width,
                "transform": src.transform * src.transform.scale(scale_factor, scale_factor)
            })

        # Generate new output filename
        new_filename = filename.replace("0.01", "0.05")  # Replace "0.01" in the filename with "0.05"
        output_file = os.path.join(output_folder, new_filename)

        # Export the result
        with rasterio.open(output_file, "w", **profile) as dst:
            dst.write(resampled_data, 1)

        print(f"File saved to: {output_file}")

print("All files have been processed.")

# End of the script

处理文件: D:\ghana\11.3\NIRv\NIRvMax_0.01_2021.tif
文件已保存到: D:\ghana\11.3\NIRv1\NIRvMax_0.05_2021.tif
处理文件: D:\ghana\11.3\NIRv\NIRvMax_0.01_2022.tif
文件已保存到: D:\ghana\11.3\NIRv1\NIRvMax_0.05_2022.tif
处理文件: D:\ghana\11.3\NIRv\NIRvMedian_0.01_2021.tif
文件已保存到: D:\ghana\11.3\NIRv1\NIRvMedian_0.05_2021.tif
处理文件: D:\ghana\11.3\NIRv\NIRvMedian_0.01_2022.tif
文件已保存到: D:\ghana\11.3\NIRv1\NIRvMedian_0.05_2022.tif
处理文件: D:\ghana\11.3\NIRv\NIRvMin_0.01_2021.tif
文件已保存到: D:\ghana\11.3\NIRv1\NIRvMin_0.05_2021.tif
处理文件: D:\ghana\11.3\NIRv\NIRvMin_0.01_2022.tif
文件已保存到: D:\ghana\11.3\NIRv1\NIRvMin_0.05_2022.tif
所有文件处理完成。
